In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Prendere in considerazione i dati caricati precedentemente e quelli relativi ai corsi d'acqua della provincia di Milano allegati al presente post. Realizzare poi un notebook che:


In [ ]:
#!pip install geopandas
#!pip install contextily
import geopandas
import contextily as ctx

In [ ]:
quartieri = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/ds964_nil_wm.zip")
bike_ciclabili = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/bike_ciclabili.zip")
fontanelle = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/Fontanelle.zip")
parchi = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/Parchi_regionali_e_nazionali_polygon/Parchi_regionali_e_nazionali_polygon.dbf")
comuni = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/Limiti01012021_g/Com01012021_g/Com01012021_g_WGS84.dbf")
province = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/Limiti01012021_g/ProvCM01012021_g/ProvCM01012021_g_WGS84.dbf")
regioni = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/Limiti01012021_g/Reg01012021_g/Reg01012021_g_WGS84.dbf")
corsi_acqua = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/Corsi_acqua_Piano_di_Gestione/Corsi_acqua_Piano_di_Gestione.dbf")

1. visualizzi su una mappa i 10 corsi d'acqua più lunghi della provincia di Milano


In [ ]:
provinciamilano = province[province.DEN_UTS == "Milano"]
corsi_provmilano = corsi_acqua[corsi_acqua.intersects(provinciamilano.unary_union)]
ax = corsi_provmilano.sort_values("SHAPE_LEN", ascending = False).head(10).to_crs(epsg = 3857).plot(figsize = (20,12))
provinciamilano.to_crs(epsg = 3857).plot(ax=ax, facecolor = "None", edgecolor = "k")
ctx.add_basemap(ax)

2. visualizzi su una mappa i Navigli e i fiumi da cui prendono l'acqua. Utilizzare colori diversi per rappresentare i navigli rispetto ai fiumi ed evidenziare bene i percorsi sulla mappa

In [ ]:
navigli = corsi_acqua[corsi_acqua.NOME.str.contains("Navigli")]
fiumi = corsi_acqua[corsi_acqua.touches(navigli.unary_union)]
ax = navigli.to_crs(epsg = 3857).plot(figsize = (20,12), line)
fiumi.to_crs(epsg=3857).plot(ax=ax, color = "r")
ctx.add_basemap(ax)

3. visualizzi su una mappa i corsi d'acqua che passano in un comune inserito dall'utente


In [ ]:
a = input("Inserisci un comune: ")
comunea = comuni[comuni.COMUNE == a]
corsicomune = corsi_acqua[corsi_acqua.crosses(comunea.geometry.squeeze())]

ax = corsicomune.to_crs(epsg = 3857).plot(figsize = (20,12), linewidth = 2)
comunea.to_crs(epsg=3857).plot(ax=ax, facecolor = "None", edgecolor = "k", linewidth = 2)
ctx.add_basemap(ax)

visuallizare solo la parte del corso d'acqua che attraversa il comune inserito dall'utente

4. visualizzi su una mappa un fiume inserito dall'utente con tutti i comuni  che attraversa. Se attraversa Milano, visualizzare solo i quartieri che attraversa


In [ ]:
b = input("Inserisci un fiume: ")
fiumea = corsi_acqua[corsi_acqua.NOME.str.contains(b)]
comunifiume = comuni[comuni.intersects(fiumea.geometry.squeeze())]
quartierifiume = quartieri[quartieri.intersects(fiumea.to_crs(epsg=4326).geometry.squeeze())]
comunifiume2 = comunifiume.drop(comunifiume[comunifiume.COMUNE == "Milano"].index)

ax = comunifiume2.to_crs(epsg = 3857).plot(figsize = (20,12), facecolor = "None", edgecolor = "k", linewidth = 2)
fiumea.to_crs(epsg=3857).plot(ax=ax, linewidth = 2)
quartierifiume.to_crs(epsg=3857).plot(ax=ax, facecolor = "None", edgecolor = "k", linewidth = 2)
ctx.add_basemap(ax)

5. visualizzi il numero di corsi d'acqua della provincia di Milano e la lunghezza totale di tutti i corsi d'acqua


In [ ]:
corsimilano = corsi_acqua[corsi_acqua.crosses(provinciamilano.geometry.squeeze())]

In [ ]:
#numero dei corsi d'acqua
corsimilano.count()["NOME"]

In [ ]:
#lunghezza totale dei corsi d'acqua
corsimilano.SHAPE_LEN.sum()

6. visualizzi su una mappa i comuni della provincia di Milano che non sono attraversati da corsi d'acqua


In [ ]:
comunimilano = comuni[comuni.within(provinciamilano.geometry.squeeze())]
comunina = comunimilano[~comunimilano.intersects(corsi_acqua.unary_union)]

ax = comunina.to_crs(epsg=3857).plot(figsize = (20,12), facecolor = "None", edgecolor = "k")
ctx.add_basemap(ax)